## To do list
    * list of files already ingested:
    
    * list of files to be ingested:
        * add Drug_target_with_primary_source.nodes.tsv

In [14]:
## Load necessary packages
import os
import pandas as pd
import glob
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

## Define the version number
version_number = "05_26_2025"
deployment_date = "2025-05-26"

In [15]:
## Notice!! Please change the file path of following codes into your own
raw_files_path = '/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/'

## Define the output path for node & edge files after formatting
# download_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/primeKG_parsed_node_{version_number}.tsv'
download_path_edge_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/AML_subset_primeKG_parsed_edge_{version_number}.tsv'

In [16]:
## Check all node files being read
## Read all BigGIM node csv file in group 1

for f in os.listdir(raw_files_path):
    if f.endswith('.tsv'):
        print(f)

primeKG_parsed_edge_05_26_2025.tsv
primeKG_parsed_node_05_19_2025.tsv
primeKG_parsed_edge_05_19_2025.tsv
primeKG_parsed_node_05_26_2025.tsv


In [17]:
## Read each individual csv files
edges_df = pd.read_csv(raw_files_path + f'primeKG_parsed_edge_{version_number}.tsv', sep='\t')

/var/folders/ml/cjwyk2ps62361rr96_3b_n080000gp/T/ipykernel_4440/617478465.py:2: DtypeWarning: Columns (4,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(raw_files_path + f'primeKG_parsed_edge_{version_number}.tsv', sep='\t')


In [18]:
edges_df.head()

,subject,object,subject_category,object_category,predicate,knowledge_souce,knowledge_level,agent_type,deploy_date,context_qualifier,id,anatomical_context_qualifier,knowledge_source,publications
0,NCBIGene:9796,NCBIGene:56992,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,2fdb36d5-77f5-54f7-8c01-2e718ca0c65d,NaN,NaN,NaN
1,NCBIGene:7918,NCBIGene:9240,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,a21015f9-12da-52d9-a964-8cfd8a8cdf36,NaN,NaN,NaN
2,NCBIGene:8233,NCBIGene:23548,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,91e4a35e-3b2a-5d96-b5b4-d67c49f3fac5,NaN,NaN,NaN
3,NCBIGene:4899,NCBIGene:11253,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,da327dc3-9a30-5907-addb-655283431e5f,NaN,NaN,NaN
4,NCBIGene:5297,NCBIGene:8601,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,9210abc4-b0af-5492-ae6a-dad0b87b7fce,NaN,NaN,NaN


In [19]:
## now find follow rows with matched MONDO code for AML
## Acute Myeloid Leukemia: 
## MONDO:0004996	childhood acute myeloid leukemia
## MONDO:0019456	acute myeloid leukemia with multilineage dysplasia
## MONDO:0015164	acute myeloid leukemia and myelodysplastic syndromes related to alkylating agent
## MONDO:0015608	acute myeloid leukemia and myelodysplastic syndromes related to radiation
## MONDO:0020317	acute myeloid leukemia with 11q23 abnormalities
## MONDO:0035112	acute myeloid leukemia with BCR-ABL1
## MONDO:0011118	bilineal acute myeloid leukemia
## MONDO:0015667	unclassified acute myeloid leukemia
## MONDO:0017893	inherited acute myeloid leukemia
## MONDO:0020078	acute myeloid leukemia with recurrent genetic anomaly
## MONDO:0019457	therapy related acute myeloid leukemia and myelodysplastic syndrome
## MONDO:0005223	acute myeloid leukemia with minimal differentiation
## MONDO:0015165	acute myeloid leukemia and myelodysplastic syndromes related to topoisomerase type 2 inhibitor
## MONDO:0015166	acute myeloid leukemia with t(8;21)(q22;q22) translocation
## MONDO:0017894	acute myeloid leukemia with CEBPA somatic mutations
## MONDO:0018256	acute myeloid leukemia with t(8;16)(p11;p13) translocation
## MONDO:0018433	acute myeloid leukemia with t(6;9)(p23;q34)
## MONDO:0018434	acute myeloid leukemia with t(9;11)(p22;q23)
## MONDO:0018435	acute myeloid leukemia with inv3(p21;q26.2) or t(3;3)(p21;q26.2)
## MONDO:0018437	acute myeloid leukemia with NPM1 somatic mutations
## MONDO:0020316	acute myeloid leukemia with abnormal bone marrow eosinophils inv(16)(p13q22) or t(16;16)(p13;q22)
## HP:0004808	Acute myeloid leukemia
## MONDO:0018436	megakaryoblastic acute myeloid leukemia with t(1;22)(p13;q13)

# List of values to search for
values = ['MONDO:0004996', 'MONDO:0019456', 'MONDO:0015164', 'MONDO:0015608', 'MONDO:0020317', 'MONDO:0035112', 'MONDO:0011118', 
'MONDO:0015667', 'MONDO:0017893', 'MONDO:0020078', 'MONDO:0019457', 'MONDO:0005223', 'MONDO:0015165', 'MONDO:0015166', 'MONDO:0017894',
'MONDO:0018256', 'MONDO:0018433', 'MONDO:0018434', 'MONDO:0018435', 'MONDO:0018437', 'MONDO:0020316', 'HP:0004808', 'MONDO:0018436']

# Filter rows where column 'a' or 'b' contains any of the values
AML_edge_df = edges_df[edges_df['subject'].isin(values) | edges_df['object'].isin(values)]
print(len(AML_edge_df))

652


In [20]:
AML_edge_df.head()

,subject,object,subject_category,object_category,predicate,knowledge_souce,knowledge_level,agent_type,deploy_date,context_qualifier,id,anatomical_context_qualifier,knowledge_source,publications
367511,DRUGBANK:DB12874,MONDO:0015667,biolink:Drug,biolink:Disease,NaN,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,ca73f159-ee5e-5128-a1e5-6980091d5d1b,NaN,NaN,NaN
367512,DRUGBANK:DB12874,MONDO:0005223,biolink:Drug,biolink:Disease,NaN,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,18f5ff45-7867-5c9b-9c28-9366875c0875,NaN,NaN,NaN
367514,DRUGBANK:DB12874,MONDO:0015165,biolink:Drug,biolink:Disease,NaN,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,f5769257-cd66-5984-a766-27950a07a99c,NaN,NaN,NaN
367515,DRUGBANK:DB12874,MONDO:0015166,biolink:Drug,biolink:Disease,NaN,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,e2b2736e-1d5f-5b17-818b-399ffb104fa2,NaN,NaN,NaN
367516,DRUGBANK:DB12874,MONDO:0017893,biolink:Drug,biolink:Disease,NaN,PrimeKG,knowledge_assertion,automated_agent,2025-05-26,NaN,5e815efa-1620-552e-9d83-20b84419502c,NaN,NaN,NaN


In [21]:
pd.set_option('display.max_rows', None)
# Optional: Reset the option to default after use
# pd.reset_option('display.max_rows')

In [22]:
AML_edge_subset_df = AML_edge_df[['subject', 'object', 'predicate']]
AML_edge_subset_df

,subject,object,predicate
367511,DRUGBANK:DB12874,MONDO:0015667,NaN
367512,DRUGBANK:DB12874,MONDO:0005223,NaN
367514,DRUGBANK:DB12874,MONDO:0015165,NaN
367515,DRUGBANK:DB12874,MONDO:0015166,NaN
367516,DRUGBANK:DB12874,MONDO:0017893,NaN
367517,DRUGBANK:DB12874,MONDO:0017894,NaN
367518,DRUGBANK:DB12874,MONDO:0018256,NaN
367519,DRUGBANK:DB12874,MONDO:0018433,NaN
367520,DRUGBANK:DB12874,MONDO:0018434,NaN
367521,DRUGBANK:DB12874,MONDO:0018435,NaN
